# Zad 3

Dane są tabele Bufor(ID, AdresUrl, OstatnieWejscie), Historia(ID, AdresUrl, OstatnieWejscie) oraz Parametry(nazwa, wartosc). Znaczenie pól w tabelach Bufor i Historia jest następujące: 

> • ID — identyfikator wiersza w tabeli,

> • AdresUrl — adres strony WWW, 

> • OstatnieWejscie — moment ostatnich odwiedzin określony z dokładnością do sekundy. 

W tabeli Parametry przechowujemy tylko jedną parę (max cache, maksymalny rozmiar bufora). W momencie wstawienia nowego wiersza do tabeli Bufor powinny być spełnione następujące postulaty: 

> • Jeśli w buforze istnieje strona o adresie, który wstawiamy, należy tylko zmodyfikować czas jej ostatniego wejścia,

> • Jeśli w buforze nie ma wstawianej strony, sprawdzamy, czy liczba wierszy przed wstawieniem jest mniejsza niż ta określona przez parametr max cache z tabeli Parametry. Mamy dwa przypadki: 
>> – Jeśli jest mniejsza, wstawiamy wiersz i kończymy.    
>> – Jeśli nie jest mniejsza, szukamy strony WWW, która ma najstarsze ostatnie wejście. Znaleziony wiersz przenosimy do tabeli Historia (gdyby się zdarzyło, że wierszy jest więcej niż jeden, wybieramy jeden z nich), ale tylko w przypadku, gdy danej strony w Historii jeszcze nie ma (jak jest, aktualizujemy tylko ostatnie wejście). 

Powyższą funkcjonalność należy zrealizować za pomocą wyzwalaczy 

In [1]:
-- create tables
DROP TABLE IF EXISTS Bufor, Historia, Parametry
GO

CREATE TABLE Bufor(
    ID INT IDENTITY PRIMARY KEY, 
    AdresUrl VARCHAR(255), 
    OstatnieWejscie DATETIME
)

CREATE TABLE Historia(
    ID INT IDENTITY PRIMARY KEY, 
    AdresUrl VARCHAR(255), 
    OstatnieWejscie DATETIME
)

CREATE TABLE Parametry(
    nazwa VARCHAR(255), 
    wartosc INT
)

INSERT INTO Parametry
VALUES ('max_cache', 4)

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.121

In [45]:
DROP TRIGGER IF EXISTS TR_doHistorii
GO

-- wyzwalacz "zamiast" na INSERT do Bufora
CREATE TRIGGER TR_doHistorii ON Bufor INSTEAD OF INSERT
AS
BEGIN
    -- pobierz wstawiony wiersz 
    DECLARE @in_AdresUrl VARCHAR(255), @in_OstatnieWejscie DATETIME
    SELECT 
        @in_AdresUrl = AdresUrl, 
        @in_OstatnieWejscie = OstatnieWejscie
    FROM 
        INSERTED
    
    -- Jeśli w buforze istnieje strona o adresie, który wstawiamy, należy tylko zmodyfikować czas jej ostatniego wejścia
    IF (EXISTS (SELECT 1 FROM Bufor WHERE AdresUrl = @in_AdresUrl))
        UPDATE Bufor 
        SET OstatnieWejscie = @in_OstatnieWejscie
        WHERE AdresUrl = @in_AdresUrl
    ELSE
    BEGIN
        -- Jeśli w buforze nie ma wstawianej strony, sprawdzamy, czy liczba wierszy przed wstawieniem jest mniejsza niż ta określona przez parametr max cache z tabeli Parametry
        DECLARE @bufor_wierszy INT = (SELECT COUNT(*) FROM Bufor)
        DECLARE @bufor_limit INT = (SELECT TOP 1 wartosc FROM Parametry)

        -- Jeśli jest mniejsza, wstawiamy wiersz i kończymy. 
        IF (@bufor_wierszy < @bufor_limit)
            INSERT INTO Bufor 
                VALUES(@in_AdresUrl, @in_OstatnieWejscie)
        ELSE
        BEGIN
            -- przenosimy najstarszy wpis do historii
            DECLARE @temp_ID INT, @temp_AdresUrl VARCHAR(255), @temp_OstatnieWejscie DATETIME
            -- wybieramy pierwszy z możliwych
            SELECT TOP 1
                @temp_ID = ID,
                @temp_AdresUrl = AdresUrl, 
                @temp_OstatnieWejscie = OstatnieWejscie
            FROM 
                Bufor
            ORDER BY OstatnieWejscie

            -- wpis jest już w historii
            IF (EXISTS (SELECT 1 FROM Historia WHERE AdresUrl = @temp_AdresUrl))
                UPDATE Historia
                SET OstatnieWejscie = @in_OstatnieWejscie
                WHERE AdresUrl = @temp_AdresUrl
            ELSE
                INSERT INTO Historia
                VALUES(@temp_AdresUrl, @temp_OstatnieWejscie)

            -- usuwamy najstarszy wpis i dodajemy nowy do bufora
            DELETE FROM Bufor WHERE ID = @temp_ID
            INSERT INTO Bufor 
                VALUES(@in_AdresUrl, @in_OstatnieWejscie)
        END
    END
END


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.106

Testy

In [67]:
INSERT INTO Bufor VALUES('test.com', '2022-03-21 15:13:00')

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.060

In [47]:
INSERT INTO Bufor VALUES('google.com', '2022-03-25 15:23:00')

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.073

In [65]:
INSERT INTO Bufor VALUES('youtube.com', '2022-03-24 07:26:00')

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.065

In [69]:
INSERT INTO Bufor VALUES('example.com', '2022-03-22 05:37:00')

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.058

In [60]:
INSERT INTO Bufor VALUES('skos.ii.uni.wroc.pl', '2022-03-24 23:47:00')

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.058

In [70]:
SELECT * FROM Bufor
SELECT * FROM Historia

(4 rows affected)

(2 rows affected)

Total execution time: 00:00:00.054

ID,AdresUrl,OstatnieWejscie
1,test.com,2022-03-21 15:13:00.000
2,google.com,2021-03-25 15:23:00.000
5,skos.ii.uni.wroc.pl,2021-03-24 23:47:00.000
8,example.com,2021-03-22 05:37:00.000


ID,AdresUrl,OstatnieWejscie
1,example.com,2021-03-24 07:26:00.000
2,youtube.com,2021-03-22 05:37:00.000


In [71]:
DROP TABLE IF EXISTS Bufor, Historia, Parametry
GO

DROP TRIGGER IF EXISTS TR_doHistorii
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.161